Making a LeNet 5 CNN Model on Fashion MNIST Dataset using Pytorch

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch import optim
from torchsummary import summary


In [2]:
training=datasets.FashionMNIST(root="data",train=True,download=True,transform=ToTensor())
testing=datasets.FashionMNIST(root="data",train=False,download=True,transform=ToTensor())

In [3]:
train_dataloader=DataLoader(training,batch_size=64)
test_dataloader=DataLoader(testing,batch_size=64)

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.c1=nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5,stride=1,padding=2)
        self.s2=nn.AvgPool2d(kernel_size=2,stride=2)
        self.c3=nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5,stride=1,padding=0)
        self.s4=nn.AvgPool2d(kernel_size=2,stride=2)
        self.c5=nn.Conv2d(in_channels=16,out_channels=120,kernel_size=5,stride=1)
        self.flatten= nn.Flatten(start_dim=1)
        self.l1=nn.Linear(in_features=120,out_features=84)
        self.l2=nn.Linear(in_features=84,out_features=10)
        self.act=nn.Tanh()
    def forward(self,x):
        x=self.c1(x)
        x=self.act(x)
        x=self.s2(x)
        x=self.c3(x)
        x=self.act(x)
        x=self.s4(x)
        x=self.c5(x)
        x=self.act(x)
        x=self.flatten(x)
        x=self.l1(x)
        x=self.act(x)
        x=self.l2(x)
        return x
        
    



In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model=NeuralNetwork().to(device)
learning_rate=0.001


In [7]:
summary(model,input_size=(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
              Tanh-8            [-1, 120, 1, 1]               0
           Flatten-9                  [-1, 120]               0
           Linear-10                   [-1, 84]          10,164
             Tanh-11                   [-1, 84]               0
           Linear-12                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
---------------------------------

In [8]:
loss_fun=nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [10]:
for i in range(100):
    for i,(data,target) in enumerate(train_dataloader):
        data=data.to(device)
        target=target.to(device)
        scores=model(data)
        loss=loss_fun(scores,target)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    accuracy_check(train_dataloader,model)
        



0.830133318901062
0.8588833212852478
0.8701666593551636
0.8791166543960571
0.8860999941825867
0.8914166688919067
0.8973666429519653
0.9023833274841309
0.9057333469390869
0.9092000126838684
0.9120500087738037
0.9146333336830139
0.9176166653633118
0.9215166568756104


In [9]:
def accuracy_check(loader,model):
    correct=0
    total=0
    model.eval()
    with torch.no_grad():
        for (x,y) in (loader):
            x=x.to(device=device)
            y=y.to(device=device)
            scores=model(x)
            scores=nn.functional.softmax(scores,dim=1)
            _,predictions=scores.max(1)
            correct+=(predictions==y).sum()
            total+= predictions.size(0)
    print(float(correct/total))